# Multi Speaker Speech to Text

Alle notwendige Pakete importieren (siehe requirements.txt)

In [ ]:
from pyannote.audio import Pipeline
from pydub import AudioSegment
import whisper
import os

Definieren welches Audio-Recording genutzt werden soll.

In [ ]:
audio_file = "ldn-topic.wav"

## Initialize pipeline

speaker-diarization pipeline initialisieren. Dafuer wird eine Registierung bei dem Model und ein Access Token fuer Huggingface benoetigt:

1. visit hf.co/pyannote/speaker-diarization and hf.co/pyannote/segmentation and accept user conditions (only if requested)
2. visit hf.co/settings/tokens to create an access token (only if you had to go through 1.)

Huggingface Token muss in Umgebungsvariable `HF_TOKEN` gesetzt werden

In [ ]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=os.environ["HF_TOKEN"])

## Audio "parsen"

Audio file parsen. `diarization` beinhaltet die abschnitte und den Sprecher

In [ ]:
diarization = pipeline(audio_file, num_speakers=2)

## Whisper Model laden

In [ ]:
whispermodel = whisper.load_model("base")

## Audio splitten und chunks zu text umwandeln

In [ ]:
audio = AudioSegment.from_wav(audio_file)
i = 1

f = open("transscript.txt", "a")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    newAudio = audio[turn.start*1000:turn.end*1000]
    newAudio.export(f"part{i}.wav", format="wav")
    result = whispermodel.transcribe(f"part{i}.wav")
    f.write(f'{speaker}:{result["text"]}\n')
    print(f"start={turn.start} stop={turn.end} speaker_{speaker}")
    i = i + 1

f.close()